In [12]:
import pandas as pd
import csv
import re
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 300
pd.options.display.max_rows = 300

%matplotlib inline

In [13]:
from modules import rename_dict as labels_names

In [14]:
df = pd.read_msgpack("../data/full_df.msgpack")

In [5]:
df["eth_group_to_code"].unique()

array(['тибетец', 'монгол', 'помор', 'калмык', 'украинец', 'кавказец',
       'сибиряк', 'жид', 'славянин', 'горец', 'великоросс', 'другой',
       'чукча', 'еврей', 'вепс', 'пермяк', 'цыган', 'узбек', 'тува',
       'грузин', 'азиат', 'хальмг', 'швед', 'татарин', 'русский', 'росс',
       'россиянин', 'крымчанин', 'чеченец', 'русскоязычный', 'немец',
       'москаль', 'поляк', 'австриец', 'казак', 'азербайджанец',
       'армянин', 'эскимос', 'таджик', 'черножопый', 'хохол', 'коми',
       'нерусь', 'уралец', 'басурманин', 'хач', 'дагестанец', 'карел',
       'осетин', 'итальянец', 'ингуш', 'перс', 'дигорец', 'грек', 'турок',
       'беларус', 'чурка', 'американец', 'казах', 'вайнах', 'ойрат',
       'абрек', 'адыг', 'кабардинец', 'узкоглазый', 'эстонец', 'русин',
       'монголоид', 'афганец', 'манси', 'японец', 'молдаванин', 'мариец',
       'бурят', 'европеец', 'западенец', 'аварец', 'кацап', 'француз',
       'гуцул', 'китаец', 'лезгин', 'латыш', 'ненец', 'прибалт',
       'испане

Можно аггрегировать по этноязыковым критериям: тюрки, славяне, персы и др.

Важно, что укрупнение должно строиться относительно взгляда россиян на них. Для россиян есть 
* народы из постсоветского пространства
    * Народы Кавказа
        * кавказец
        * горец
        * грузин
        * чеченец
        * азербайджанец
        * армянин
        * черножопый (и в средней Азии)
        * хач
        * дагестанец
        * осетин
        * ингуш
        * дигорец
        * чурка (?)
        * вайнах
        * абрек?
        * адыг
        * кабардинец
        * аварец
        * лезгин
        * алан
        * даргинец
        * черкес
        * балкарец
        * карачаевец
        * кударец
        * лакец
        * абазин
        * абхаз
        * сван
        * мегрел
    * Тюркские народы до Урала
        * калмык
        * хальмг
        * татарин
        * ойрат
        * чуваш
        * башкир
        * кумык (или Кавказ)
        * ногаец (или Кавказ)
    * Прибалты (Балтийцы + Прибалтийско-финские народы)
        * эстонец
        * латыш
        * прибалт
        * литовец
        * карел
        * вепс
        * помор?
        * финн
    * Фино-угры
        * мариец
        * хант
        * манси
        * черемис
        * угр
        * удмурт
        * чухонец
        * саам
        * чудь
        * мордва
        * эрзя
        * мокша
    * Монголоиды за Уралом
        * тува
        * коми
        * монголоид
        * бурят
        * хакас
        * тунгус
        * алтаец
        * шорец
    * Дальний Восток
        * камчадал
        * нанаец
        * тунгус (Монголоиды за Уралом)
        * эвенк
        * коряк
        * алеут
    * Народы Крайнего Севера?
        * ненец
        * чукча (Дальний Восток)
        * эскимос (Дальний Восток)
        * якут (Монголоиды за Уралом, Дальний Восток)
    * Южные славяне, балканы
        * молдаванин
        * румын
        * македонец
        * серб
        * болгарин
        * албанец
        * мамалыжник
        * босниец
        * хорват
        * словак
        * черногорец
        * словенец
    * Западные славяне
        * поляк
        * чех
    * Восточные славяне
        * великоросс
        * украинец
        * русский
        * казак
        * хохол
        * москаль
        * росс
        * беларус
        * русин
        * западенец
        * кацап
        * гуцул
        * русня
        * малорос
        * салоед
        * русак
    * Среднеазиаты
        * узбек
        * таджик
        * чурка (и в кавказе)
        * казах
        * киргиз
        * туркмен
        * уйгур
        * чучмек (и в кавказе)
        * среднеазиат
        * джамшан
    *
* Народы не из РФ
    * «Коренная» Европа
        * немец
        * австриец
        * итальянец
        * грек
        * европеец
        * француз
        * испанец
        * швед
        * норвежец
        * англичанин
        * венгр (или угры)
        * мадьяр (или угры)
        * скандинав
        * ирландец
        * исландец
        * португалец
        * швейцарец
        * голландец
        * датчанин
        * бельгиец
        * киприот
        * фламандец
        * шотландец
    * Новый Свет
        * американец
        * мексиканец
        * латиноамериканец
        * канадец
        * кубинец
        * новозеландец
        * австралиец
        * перуанец
        * бразилец
        * аргентинец
    * Ближний Восток
        * перс
        * сириец
        * израильтянин
        * турок
        * курд
        * осман
        * иранец
        * ассириец
        * еврей
        * ливанец
        * палестинец
        * египтянин
        * фарси
        * ливиец
        * саудовец
        * пуштун
        * иракец
        * жид
        * ашкенази
    * Восточная Азия
        * японец
        * китаец
        * кореец
        * южнокореец
        * маньчжур
        * северокореец
        * хань
        * монгол
    * Юго-Восточная Азия
        * вьетнамец
        * таец
        * филиппинец
        * малаец
    * Южная Азия
        * пакистанец (Большой Ближний Восток)
        * индиец
        * афганец (Большой Ближний Восток)
        * тибетец
    * Африка
        * эфиоп
        * алжирец (Большой Ближний Восток)
        * бербер
        * африканец
        * негр
        * зулус
        * темнокожий (не только в африке)
* Исторические
    
* Непонятно
    * сибиряк
    * славянин
    * другой
    * пермяк — нет
    * цыган
    * азиат
    * россиянин
    * крымчанин
    * русскоязычный
    * нерусь
    * уралец
    * басурманин
    * узкоглазый
    * варяг
    * инородец
    * евразиец
    * кет
    * ксенофобия
    * папуас
    * гастарбайтер
    * нацмен
    * восточноевропеец
    * метис
    * караим
    * гагауз
    * тюрк
    * ариец
    * северянин
    * мулат
    * абориген
    * южанин

In [27]:
eth_groups = {
    "Народы Кавказа": [
        "кавказец",
        "горец",
        "грузин",
        "чеченец",
        "азербайджанец",
        "армянин",
        "хач",
        "дагестанец",
        "осетин",
        "ингуш",
        "дигорец",
        "чурка",
        "вайнах",
        "абрек",
        "адыг",
        "кабардинец",
        "аварец",
        "лезгин",
        "алан",
        "даргинец",
        "черкес",
        "балкарец",
        "карачаевец",
        "кударец",
        "лакец",
        "абазин",
        "абхаз",
        "сван",
        "мегрел"],
    "Тюркские народы до Урала": [
        "калмык",
        "хальмг",
        "татарин",
        "ойрат",
        "чуваш",
        "башкир",
        "кумык",
        "ногаец"],
    "Прибалты": [
        "эстонец",
        "латыш",
        "прибалт",
        "литовец",
        "карел",
        "вепс",
        "помор",
        "финн"],
    "Фино-угры": [
        "мариец",
        "хант",
        "манси",
        "черемис",
        "угр",
        "удмурт",
        "чухонец",
        "саам",
        "чудь",
        "мордва",
        "эрзя",
        "мокша"],
    "Монголоиды за Уралом": [
        "тува",
        "коми",
        "монголоид",
        "бурят",
        "хакас",
        "тунгус",
        "алтаец",
        "шорец"],
    "Дальний Восток": [
        "камчадал",
        "нанаец",
        "тунгус",
        "эвенк",
        "коряк",
        "алеут"],
    "Народы Крайнего Севера": [
        "ненец",
        "чукча",
        "эскимос",
        "якут"],
    "Южные славяне, балканы": [
        "молдаванин",
        "румын",
        "македонец",
        "серб",
        "болгарин",
        "албанец",
        "мамалыжник",
        "босниец",
        "хорват",
        "словак",
        "черногорец",
        "словенец"],
    "Западные славяне": [
        "поляк",
        "чех"],
    "Восточные славяне": [
        "великоросс",
        "украинец",
        "русский",
        "казак",
        "хохол",
        "москаль",
        "росс",
        "беларус",
        "русин",
        "западенец",
        "кацап",
        "гуцул",
        "русня",
        "малорос",
        "салоед",
        "русак"],
    "Среднеазиаты": [
        "узбек",
        "таджик",
        "чурка",
        "казах",
        "киргиз",
        "туркмен",
        "уйгур",
        "чучмек",
        "среднеазиат",
        "джамшан"],
    "«Коренная» Европа": [
        "немец",
        "австриец",
        "итальянец",
        "грек",
        "европеец",
        "француз",
        "испанец",
        "швед",
        "норвежец",
        "англичанин",
        "венгр",
        "мадьяр",
        "скандинав",
        "ирландец",
        "исландец",
        "португалец",
        "швейцарец",
        "голландец",
        "датчанин",
        "бельгиец",
        "киприот",
        "фламандец",
        "шотландец"],
    "Новый Свет": [
        "американец",
        "мексиканец",
        "латиноамериканец",
        "канадец",
        "кубинец",
        "новозеландец",
        "австралиец",
        "перуанец",
        "бразилец",
        "аргентинец"],
    "Большой Ближний Восток": [
        "перс",
        "сириец",
        "израильтянин",
        "турок",
        "курд",
        "осман",
        "иранец",
        "ассириец",
        "еврей",
        "жид", # этнофолизм
        "ашкенази",
        "ливанец",
        "палестинец",
        "египтянин",
        "фарси",
        "ливиец",
        "саудовец",
        "пуштун",
        "иракец",
        "пакистанец",
        "афганец",
        "алжирец"],
    "Восточная Азия": [
        "японец",
        "китаец",
        "кореец",
        "южнокореец",
        "маньчжур",
        "северокореец",
        "хань",
        "монгол"],
    "Юго-Восточная Азия": [
        "вьетнамец",
        "таец",
        "филиппинец",
        "малаец"],
    "Южная Азия": [
        "индиец",
        "тибетец"],
    "Африка": [
        "эфиоп",
        "бербер",
        "африканец",
        "негр",
        "зулус",
        "темнокожий"],
    "Другое": [
        "сибиряк",
        "славянин",
        "другой",
        "пермяк",
        "цыган",
        "азиат",
        "россиянин",
        "крымчанин",
        "русскоязычный",
        "нерусь",
        "уралец",
        "басурманин",
        "узкоглазый",
        "варяг",
        "инородец",
        "евразиец",
        "кет",
        "ксенофобия",
        "папуас",
        "гастарбайтер",
        "нацмен",
        "восточноевропеец",
        "метис",
        "караим",
        "гагауз",
        "тюрк",
        "ариец",
        "северянин",
        "мулат",
        "абориген",
        "южанин",
        "черножопый"
    ]
}


In [28]:
dict2replace = {eth: group  for group, eth_list in eth_groups.items() for eth in eth_list}

In [32]:
df["aggregated_groups"] = df["eth_group_to_code"].replace(dict2replace)

In [35]:
df["aggregated_groups"].unique()

array(['Южная Азия', 'Восточная Азия', 'Прибалты',
       'Тюркские народы до Урала', 'Восточные славяне', 'Народы Кавказа',
       'Другое', 'Большой Ближний Восток', 'Народы Крайнего Севера',
       'Среднеазиаты', 'Монголоиды за Уралом', '«Коренная» Европа',
       'Западные славяне', 'Новый Свет', 'Фино-угры',
       'Южные славяне, балканы', 'Африка', 'Дальний Восток',
       'Юго-Восточная Азия'], dtype=object)

In [33]:
df.head()

,about_whole_nation_recoded_value,about_whole_nation_recoded_label,assessor,comment,date,document.id,do_text_make_sense_raw_value,do_text_make_sense_raw_label,encourage_aggression_meaning_value,encourage_aggression_meaning_label,eth_group_to_code,has_eth_conflict_raw_value,has_eth_conflict_raw_label,has_ethnonym_raw_value,has_ethnonym_raw_label,has_pos_eth_interaction_raw_value,has_pos_eth_interaction_raw_label,has_topic_culture_value,has_topic_culture_label,has_topic_daily_routine_value,has_topic_daily_routine_label,has_topic_economics_value,has_topic_economics_label,has_topic_ethicity_value,has_topic_ethicity_label,has_topic_history_value,has_topic_history_label,has_topic_humour_value,has_topic_humour_label,has_topic_migration_value,has_topic_migration_label,has_topic_other_value,has_topic_other_label,has_topic_politics_value,has_topic_politics_label,has_topic_religion_value,has_topic_religion_label,has_topic_society_social_value,has_topic_society_social_label,is_ethicity_aggressor_meaning_value,is_ethicity_aggressor_meaning_label,is_ethicity_dangerous_meaning_value,is_ethicity_dangerous_meaning_label,is_ethicity_superior_meaning_value,is_ethicity_superior_meaning_label,is_text_neg_raw_value,is_text_neg_raw_label,is_text_positive_raw_value,is_text_positive_raw_label,opinion_about_ethnonym_recoded_value,opinion_about_ethnonym_recoded_label,represent_ethicity_meaning_value,represent_ethicity_meaning_label,seed_eth_group,source_x,stage,source_y,source_text,lemm_text,city,region,created_date,instance_id,aggregated_groups
0,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_тибетец,Южная Азия
1,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол,Восточная Азия
2,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_тибетец,Южная Азия
3,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол,Восточная Азия
4,NaN,NaN,Oleg Nagornyy,NaN,2016-07-29 15:10:42,883,1,yes,NaN,NaN,монгол,0.0,no,1.0,one,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,NaN,NaN,NaN,NaN,NaN,2.0,strong,0.0,no,NaN,NaN,NaN,NaN,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол,Восточная Азия


In [34]:
df.to_msgpack("../data/full_df.msgpack")